# **C23-PR487 ~ LapakIn**

> Assumption
- Successful MSMEs defined by:  
Rating >= 4 and Review >= 6 on Google Maps 

# Setup

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
df_raw = pd.read_csv('successful_msme.csv')

# EDA & Preprocessing

In [ ]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 635 entries, 0 to 634
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   url                      635 non-null    object 
 1   nama toko                635 non-null    object 
 2   rating                   635 non-null    float64
 3   jumlah review            635 non-null    int64  
 4   kategori                 635 non-null    object 
 5   alamat                   635 non-null    object 
 6   kota                     635 non-null    object 
 7   jumlah mall terdekat     635 non-null    int64  
 8   jumlah kantor terdekat   635 non-null    int64  
 9   jumlah sekolah terdekat  635 non-null    int64  
 10  nama_mall                494 non-null    object 
 11  nama_kantor              454 non-null    object 
 12  nama_sekolah             581 non-null    object 
 13  label                    635 non-null    object 
dtypes: float64(1), int64(4), o

In [ ]:
df = df_raw[['jumlah mall terdekat', 'jumlah kantor terdekat', 'jumlah sekolah terdekat', 'label']]

In [ ]:
df.head()

,jumlah mall terdekat,jumlah kantor terdekat,jumlah sekolah terdekat,label
0,1,4,8,toko_kopi
1,6,7,2,toko_kopi
2,3,1,10,toko_kopi
3,0,4,2,toko_kopi
4,1,4,3,toko_kopi


In [ ]:
df.describe()

,jumlah mall terdekat,jumlah kantor terdekat,jumlah sekolah terdekat
count,635.000000,635.000000,635.000000
mean,2.508661,2.259843,5.132283
std,2.480429,2.364340,3.459748
min,0.000000,0.000000,0.000000
25%,1.000000,0.000000,2.000000
50%,2.000000,2.000000,5.000000
75%,4.000000,4.000000,7.000000
max,13.000000,12.000000,16.000000


## Check Duplicate

In [ ]:
df_raw.duplicated().sum()

0

## Check Null Value

In [ ]:
df.isnull().sum()

jumlah mall terdekat       0
jumlah kantor terdekat     0
jumlah sekolah terdekat    0
label                      0
dtype: int64

## Check Outlier

In [ ]:
def compute_reasonable_bound(selected_attribute):
    Q1 = selected_attribute.quantile(0.25)
    Q3 = selected_attribute.quantile(0.75)
    IQR = Q3-Q1
    return [Q3 + (1.5*IQR), Q1 - (1.5*IQR)]
    
def compute_outliers(attribute, dataset):
    selected_attribute = dataset.sort_values(by=[attribute])[attribute]
    [RUB, RLB] = compute_reasonable_bound(selected_attribute)
    return len(dataset[(dataset[attribute] > RUB) | (dataset[attribute] < RLB)])

def check_outlier(dataset):
    attributes = []
    outliers = []
    for attribute in df:
        if dataset[attribute].dtype != 'O':
            many_outliers = compute_outliers(attribute, dataset)
            percentage_of_outlier = (many_outliers/len(dataset[attribute])) * 100
            attributes.append(attribute)
            outliers.append(percentage_of_outlier)

    data = {'attribute': attributes, 'percentage of outlier (%)': outliers}
    outlier_df = pd.DataFrame(data)
    return outlier_df

In [ ]:
check_outlier(df)

,attribute,percentage of outlier (%)
0,jumlah mall terdekat,4.409449
1,jumlah kantor terdekat,0.157480
2,jumlah sekolah terdekat,0.629921


## Encode Label

In [ ]:
label_encoder = preprocessing.LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

<ipython-input-139-6302bf2ea4e7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = label_encoder.fit_transform(df['label'])


In [ ]:
list(label_encoder.inverse_transform([0, 1, 2, 3, 4]))

['toko_kopi', 'usaha_baju', 'usaha_fotokopi', 'usaha_laundry', 'usaha_makanan']

## Split Data

In [ ]:
X = df.drop(['label'],axis=1)
y = df['label']

In [ ]:
# Get 80% of the dataset as the training set. Put the remaining 20% as cross validation set
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.80, random_state=1)

## Normalize

In [ ]:
X_train.describe()

,jumlah mall terdekat,jumlah kantor terdekat,jumlah sekolah terdekat
count,127.000000,127.000000,127.000000
mean,2.267717,2.157480,5.251969
std,2.424900,2.385223,3.431804
min,0.000000,0.000000,0.000000
25%,0.500000,0.000000,2.000000
50%,2.000000,1.000000,5.000000
75%,3.000000,3.000000,8.000000
max,13.000000,12.000000,13.000000


In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

columns = ['jumlah mall terdekat', 'jumlah kantor terdekat', 'jumlah sekolah terdekat']
X_train = pd.DataFrame(scaler.transform(X_train), columns=columns)
X_cv = pd.DataFrame(scaler.transform(X_cv), columns=columns)

In [ ]:
X_train.head()

,jumlah mall terdekat,jumlah kantor terdekat,jumlah sekolah terdekat
0,0.717202,0.354624,0.511380
1,-0.110840,-0.487193,2.266655
2,-0.110840,-0.908102,0.218834
3,-0.938883,-0.908102,1.096472
4,1.545245,-0.908102,-1.243896


In [ ]:
X_cv.head()

,jumlah mall terdekat,jumlah kantor terdekat,jumlah sekolah terdekat
0,1.959266,0.775532,1.389018
1,-0.938883,-0.908102,-1.243896
2,1.545245,-0.066285,-1.243896
3,-0.524862,-0.487193,1.389018
4,-0.524862,2.880074,-0.658804


# Modeling

In [ ]:
model = Sequential(
    [
        Dense(100, activation = 'relu'),
        Dense(50, activation = 'relu'),
        Dense(25, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(5, activation = 'linear')
    ]
)

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy']
)

model.fit(
    X_train,y_train,
    epochs=600
)

Epoch 1/600
4/4 [==============================] - 1s 5ms/step - loss: 1.6018 - accuracy: 0.2047
Epoch 2/600
4/4 [==============================] - 0s 4ms/step - loss: 1.5893 - accuracy: 0.2835
Epoch 3/600
4/4 [==============================] - 0s 4ms/step - loss: 1.5766 - accuracy: 0.3228
Epoch 4/600
4/4 [==============================] - 0s 4ms/step - loss: 1.5691 - accuracy: 0.3071
Epoch 5/600
4/4 [==============================] - 0s 8ms/step - loss: 1.5616 - accuracy: 0.3071
Epoch 6/600
4/4 [==============================] - 0s 4ms/step - loss: 1.5555 - accuracy: 0.3071
Epoch 7/600
4/4 [==============================] - 0s 4ms/step - loss: 1.5494 - accuracy: 0.3150
Epoch 8/600
4/4 [==============================] - 0s 4ms/step - loss: 1.5426 - accuracy: 0.3701
Epoch 9/600
4/4 [==============================] - 0s 6ms/step - loss: 1.5372 - accuracy: 0.3701
Epoch 10/600
4/4 [==============================] - 0s 4ms/step - loss: 1.5320 - accuracy: 0.3701
Epoch 11/600
4/4 [===========

In [165]:
def get_predicted_label(y_result):
  rows = y_result.shape[0]
  y_pred = np.zeros(rows)
  for row in range(rows):
      y_pred[row] = np.argmax(y_result[row])
  return y_pred

In [166]:
y_result = model.predict(X_train)
y_pred = get_predicted_label(y_result)
print(f"training MSE (using sklearn function): {mean_squared_error(y_train, y_pred) / 2}")

4/4 [==============================] - 0s 3ms/step
training MSE (using sklearn function): 0.5236220472440944


In [167]:
y_result = model.predict(X_cv)
y_pred = get_predicted_label(y_result)
print(f"cv MSE (using sklearn function): {mean_squared_error(y_cv, y_pred) / 2}")

16/16 [==============================] - 0s 1ms/step
cv MSE (using sklearn function): 2.324803149606299


In [168]:
X_cv_new_sample = pd.DataFrame([[2, 12, 11]], columns = ['jumlah mall terdekat', 'jumlah kantor terdekat', 'jumlah sekolah terdekat'], dtype = float)
X_cv_new_sample = scaler.transform(X_cv_new_sample)
y_result_new_sample = model.predict(X_cv_new_sample)
y_pred_new_sample = get_predicted_label(y_result_new_sample)
print(f"cv MSE (using sklearn function): {mean_squared_error([0], y_pred_new_sample) / 2}")

1/1 [==============================] - 0s 23ms/step
cv MSE (using sklearn function): 2.0
